In [6]:
from __future__ import absolute_import
import os, re
import sys, random
import yaml, time
import json
import pickle
from word2number import w2n
from EquationConverter import EquationConverter

In [7]:
data_root_dir = os.path.join(os.getcwd(), '../MWP-Automatic-Solver/data')
data_dir = os.path.join(data_root_dir, 'datasets')

In [8]:
PROBLEM_LIST = []

In [9]:
#DIR_PATH = os.path.abspath(os.path.dirname(__file__))
DIR_PATH = os.path.abspath(os.getcwd())
DIR_PATH

'/home/ubuntu/DL/MWPS/mwps'

In [10]:
USE_GENERATED = False
TEST_SPLIT = 0.05
WORDS_FOR_OPERATORS = False

# Composite list of MWPs
PROBLEM_LIST = []

# The same list with all equations converted from infix to cleaned infix
CLEAN_INFIX_CONVERTED_PROBLEM_LIST = []

# The same list with all equations converted from infix to Polish notation
POLISH_CONVERTED_PROBLEM_LIST = []

# The same list with all equations converted from infix to Reverse Polish notation
REVERSE_POLISH_CONVERTED_PROBLEM_LIST = []

# The generated data (not used in testing)
GENERATED = []

# Dataset specific
AI2 = []
ILLINOIS = []
COMMONCORE = []
MAWPS = []

KEEP_INFIX_PARENTHESIS = True
MAKE_IND_SETS = True

# Large test sets
PREFIX_TEST = []
POSTFIX_TEST = []
INFIX_TEST = []

In [11]:
settings = {
'dataset': 'train_all_postfix.p',
'duplication': False,
'test': 'postfix',
'model': False,
'layers': 2,
'heads': 8,
'd_model': 256,
'dff': 1024,
'lr': 'scheduled',
'dropout': 0.1,
'epochs': 300,
'batch': 128,
'pretrain': False,
'beta_1': 0.95,
'beta_2': 0.99,
'pos': False,
'pos_words': False,
'remove_stopwords': False,
'as_lemmas': False,
'reorder': False,
'tagging': 'lst',
'input': False,
'seed': 420365,
'save': True
}

In [12]:
SEED = settings['seed']
random.seed(SEED)

In [13]:
def one_sentence_clean(text):
    # Clean up the data and separate everything by spaces
    text = re.sub(r"(?<!Mr|Mr|Dr|Ms)(?<!Mrs)(?<![0-9])(\s+)?\.(\s+)?", " . ",
                  text, flags=re.IGNORECASE)
    text = re.sub(r"(\s+)?\?(\s+)?", " ? ", text)
    text = re.sub(r",", "", text)
    text = re.sub(r"^\s+", "", text)
    text = text.replace('\n', ' ')
    text = text.replace("'", " '")
    text = text.replace('%', ' percent')
    text = text.replace('$', ' $ ')
    text = re.sub(r"\.\s+", " . ", text)
    text = re.sub(r"\s+", ' ', text)
    
    sent = []
    for word in text.split(' '):
        try:
            sent.append(str(w2n.word_to_num(word)))
        except:
            sent.append(word)
    
    return ' '.join(sent)        

In [14]:
def to_lower_case(text):
    try:
        return text.lower()
    except:
        return text

In [16]:
def transform_AI2():
    print("\nWorking on AI2 data...")
    problem_list = []
    with open(os.path.join(data_dir, 'AI2/questions.txt'), "r") as fh:
        content = fh.readlines()
        
    for i in range(len(content)):
        if i % 3 ==0 or i == 0:
            question_text = one_sentence_clean(content[i].strip())            
            eq = content[i + 2].strip()
            
            problem = [("question", to_lower_case(question_text)),
                       ("equation", to_lower_case(eq)),
                       ("answer", content[i+1].strip())
                      ]
            
            if problem != []:
                problem_list.append(problem)
                AI2.append(problem)
        
    total_problems = int(len(content) / 3)    
    print(f"-> Retrived {len(problem_list)} / {total_problems} problems.")
    print("...done.\n")
            
    return AI2

In [17]:
def transform_all_datasets():
    total_datasets = []
    # Iteratively rework all the data
    total_datasets.append(transform_AI2())
#     total_datasets.append(transform_CommonCore())
#     total_datasets.append(transform_Illinois())
#     total_datasets.append(transform_MaWPS())
#     if USE_GENERATED:
#         total_datasets.append(transform_custom())

    return total_datasets

In [100]:
def convert_to(problem_list, tag):
    output = []
    
    for problem in problem_list:
        problem_dict = dict(problem)
        ol = []
        discard = False
        
        for k, v in problem_dict.items():
            if k == 'equation':
                convert = EquationConverter()
                convert.eqset(v)

In [18]:
print("Transforming all original datasets...")
print(f"Splitting {(1 - TEST_SPLIT) * 100}% for training.")
print("NOTE: Find resulting data binaries in the data folder.")
total_filtered_datasets = transform_all_datasets()

Transforming all original datasets...
Splitting 95.0% for training.
NOTE: Find resulting data binaries in the data folder.

Working on AI2 data...
-> Retrived 395 / 395 problems.
...done.



In [19]:
# Split
AI2_TEST = AI2[:int(len(AI2) * TEST_SPLIT)]
AI2 = AI2[int(len(AI2) * TEST_SPLIT):]

In [20]:
PROBLEM_LIST += AI2

In [74]:
AI2[0]

[('question',
  'joan has 9 blue balloons sally has 5 blue balloons and jessica has 2 blue balloons . how many blue balloons do they have in total ? '),
 ('equation', 'x = 9 + 5 + 2'),
 ('answer', '16')]

In [22]:
random.shuffle(PROBLEM_LIST)

In [23]:
test_pre_ai2 = convert_to(AI2_TEST, "prefix")

NameError: name 'convert_to' is not defined

In [87]:
equation = AI2[0][1][1]
eq = re.search(r"([a-z]+(\s+)?=|=(\s+)?[a-z]+)", equation).group(1)
eq

'x ='

In [88]:
eq = re.sub("=", "", eq)
eq

'x '

In [89]:
eq = re.sub(r"([a-z]+(\s+)?=|=(\s+)?[a-z]+)", "", eq)
eq

'x '

In [90]:
equation.replace(' ', ""), eq.replace(' ', "")

('x=9+5+2', 'x')

In [93]:
split_expression = re.findall(r"(\d*\.?\d+|[^0-9])", equation.replace(' ', ""))
split_expression

['x', '=', '9', '+', '5', '+', '2']

In [101]:
OPERATORS = {'+', '-', '*', '/', '(', ')', '^'}
PRIORITY = {'+': 2, '-': 2, '*': 3, '/': 3, '^': 4}

def __infix_to_postfix(filtered_expresseion, equation_equals):
        stack = []
        output = ""

        split_expression = re.findall(r"(\d*\.?\d+|[^0-9])", filtered_expresseion)
        for char in split_expression:
            if char not in OPERATORS:
                output += char
            elif char == '(':
                stack.append(char)
            elif char == ')':
                while stack != [] and stack[len(stack)-1] != '(':
                    output += ' '
                    output += stack.pop()
                stack.pop()
            else:
                output += ' '
                while stack != [] and stack[len(stack)-1] != '(' and PRIORITY[char] < PRIORITY[stack[len(stack)-1]]:
                    output += stack.pop()

                stack.append(char)

        while stack != []:
            output += ' '
            output += stack.pop()

        return output

In [102]:
__infix_to_postfix(equation.replace(' ', ""), eq.replace(' ', ""))

'x=9 5 2 + +'